# GOV.UK structural network adjacency
We work through a tutorial to learn about key network metrics and statistics based on this [blog post](https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python#fnref:import). 

We will be using the [GOV.UK structural network adjacency list](https://ckan.publishing.service.gov.uk/dataset/gov-uk-structural-network-adjacency-list?utm_source=ckan) data for this tutorial. This shows the connections (edges) between source and sink pages (nodes).

This tutorial will take us through the basics of network visualisations and analysis using NetworkX and Python visualisation tools. The data itself doesn't matter so much, we are learning the principles of graph theory and `networkx` functionality that 
can be generalised to any future problems you might encounter.  

For further ideas see the [list of algorithms](https://networkx.github.io/documentation/stable/reference/algorithms/index.html) available for use with networkx.

## What is GOV.UK?

[GOV.UK](https://www.gov.uk/) is the main portal to government for citizens (and non-citizens). It enables users to interact with government in a number of situations,
 from applying for their first driving licence to checking foreign travel advice before jetting off into the sun.
  GOV.UK Data Labs (a multi-disciplinary team of data scientists, developers and others) have found it fruitful to represent GOV.UK as a network. Here's a 
[blog post](https://dataingovernment.blog.gov.uk/2019/08/07/when-worlds-collide-putting-data-science-into-production/) on how this approach allowed us to use algorithms 
to automatically recommend content to users based on what page they were on. This is now live on the site manifest as related links.  

## The challenge

After completing this notebook or during it, can you formulate a research question for a thesis project? This should take the form of a hypothesis. 
We'll pick our favourite on the day and will favour bold ideas that try to make use of other data in conjunction with this (i.e. how users move around the site, seasonal patterns).
The winner will be a hypothesis that is testable and that may reduce the journey length of users on the site.
We could then discuss the possibility of collaborating on this work in the future.


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
from operator import itemgetter
import community
import networkx as nx
import numpy as np
import os
import pandas as pd

from holoviews.element.graphs import layout_nodes
import holoviews as hv
import holoviews.operation.datashader as hd

# Enable multiline outputs in this notebook
InteractiveShell.ast_node_interactivity = "all"

# Enable interactivity for plots
hv.extension("bokeh")

## Reading the data
Rather than use the method of reading in the data used in the Quaker tutorial, we will opt for an approach using pandas.

Make sure to [download the data]((https://ckan.publishing.service.gov.uk/dataset/gov-uk-structural-network-adjacency-list?utm_source=ckan)), and place it in the `data/processed_network` folder at the parent-level of this repository. Alternatively, read in the data directly from the internet.

In [ ]:
# Define a relative path to the data
input_dir = os.path.join("..", "..", "data", "processed_network")
data_path = os.path.join(input_dir, "structural_network_adjacency_list_20190301.csv")
data_path

# Read in the data downloaded locally
df_edges = pd.read_csv(data_path)
df_edges

In [ ]:
# Define the URL to the data
data_url = "https://assets.publishing.service.gov.uk/media/5cde7160ed915d439fafebef/structural_network_adjacency_list_20190301.csv"

# Read in the data directly from the internet
df_edges = pd.read_csv(data_url)
df_edges

We now have the edges loaded in, but need a unique set of nodes for our network. Just in case there are some nodes listed in `source_base_path` but not `sink_base_path`, we need to take a unique set across both columns. 

This [SO answer](https://stackoverflow.com/a/26977495) will help us get an array of unique nodes. We will then create a pandas DataFrame, and arbitrarily set the index as `node_id`. Note we have sorted the array of unique nodes for ease here so that the main GOV.UK page slug (`/`) has node ID `0`. 

In [ ]:
# Get an array of unique nodes
unique_nodes = pd.unique(df_edges[["source_base_path", "sink_base_path"]].values.ravel("K"))

# Sort the nodes alphabetically, and artifically create a 'node_id' column using the index
df_nodes = pd.DataFrame(sorted(unique_nodes), columns=["nodes"])
df_nodes.index.name = "node_id"
df_nodes

For completeness, let's label the node IDs for both source and sink pages in `df_edges`.

Here, we merge `df_nodes` to `df_edges` for both `source_base_path` and `sink_base_path` columns. The chained `reset_index` and `rename` functions on `df_nodes` ensure we don't create duplicate source/sink slug columns during the merge, and also create unique node ID column names for the source/sink slug columns.

In [ ]:
# Add the node IDs back in for each of the sources and sinks
for col in df_edges.columns:
    if "_base_path" in col:
        df_edges = df_edges.merge(df_nodes.reset_index().rename(columns={"nodes": col, "node_id": col+"_id"}),
                                  on=col,
                                  how="left", 
                                  validate="m:1")
df_edges

## Creating the graph
In NetworkX, we can add two lists/arrays of nodes, and edges into a single network object that understands how nodes and edges are related. 

This object is called a Graph, referring to one of the common terms for data organized as a network \[N.B. it does not refer to any visual representation of the data. Graph here is used purely in a mathematical, network analysis sense\]. First you must initialize a Graph object with the following command:

In [ ]:
# Instantiate a NetworkX Graph object
G = nx.Graph()

Note this is an undirected graph. However, the data is about how pages are linked together (hence the source and sink pages!), so a direct graph (digraph) is probably makes more sense. For our demonstration purposes, let's leave it as an undirected graph.

We can add some attribute data to the graph, to help us remember what it's about

In [ ]:
G = nx.Graph(name="Graph using GOV.UK structural network adjacency list data", 
             query_used="appropriate descriptive name")
G.graph

We can then generate a list of nodes (`node_names`), and a list of edges (`edges`) from our pandas DataFrames:

In [ ]:
# Extract the slugs for each node
node_names = df_nodes.nodes.values
node_names

In [ ]:
# Get the source and destination ('sink') slugs
edges = df_edges[["source_base_path", "sink_base_path"]].values
edges

And add these to the digraph:

In [ ]:
# Add nodes and edges to 'G'
G.add_nodes_from(node_names)
G.add_edges_from(edges)

In [ ]:
# Print some info about 'G'
print(nx.info(G))

This is a quick way of getting some general information about your graph, but as you’ll learn in subsequent sections, it is only scratching the surface of what NetworkX can tell you about your data.

## Adding Attributes
For NetworkX, a Graph object is one big thing (your network) made up of two kinds of smaller things (your nodes and your edges). So far you’ve uploaded nodes and edges (as pairs of nodes), but NetworkX allows you to add attributes to both nodes and edges, providing more information about each of them.

Later on in this tutorial, you’ll be running metrics and adding some of the results back to the Graph as attributes. For now, let’s make sure your Graph contains all of the attributes that are currently in our CSV.

You’ll want to return to a DataFrames you created at the beginning of your script: `df_nodes` and `df_edges`. These contain useful information, such as the node ID and the edge link type, which you'll want to add to our graph.

There are a couple ways to do this, but NetworkX provides two convenient functions for adding attributes to all of a Graph’s nodes or edges at once: `nx.set_node_attributes()` and `nx.set_edge_attributes()`. 

To use these functions, you’ll need your attribute data to be in the form of a Python dictionary, in which node names are the keys and the attributes you want to add are the values. You’ll want to create a dictionary for each one of your attributes, and then add them using the functions above.

In [ ]:
# Just to recall
df_nodes.head()

In [ ]:
# Create a dictionary from the nodes and node IDs (index of 'df_nodes')
node_id_dict = dict(zip(df_nodes.nodes, df_nodes.index))
node_id_dict["/"]

In [ ]:
# To create the edge attributes, the key needs to be a tuple of source and sink slugs
edge_keys = list(zip(df_edges.source_base_path, df_edges.sink_base_path))
edge_keys[0:2]

# Then create the dictionary with link type as the values
edge_link_type_dict = dict(zip(edge_keys, df_edges.link_type))
edge_link_type_dict[("/1619-bursary-fund", "/courses-qualifications")]

Now we created the attribute dictionaries, let's add them to the graph:

In [ ]:
# Add the node and edge attributes
nx.set_node_attributes(G, node_id_dict, "node_id")
nx.set_edge_attributes(G, edge_link_type_dict, "link_type")

Now all of your nodes have these attributes, and you can access them at any time. For example, you can print out all the node_ids of your nodes by looping through them and accessing the node_id attribute, like this (note we've limited this to the first ten nodes):

In [ ]:
# Print the node name and ID for the first ten nodes
for ix, n in enumerate(G.nodes()):
    if ix < 10:
        print("Node: '{0}'; Node ID: '{1}'".format(n, G.node[n]["node_id"]))
    else:
        break

Now you’ve learned how to create a Graph object and add attributes to it. In the next section, you’ll learn about a variety of metrics available in NetworkX and how to access them. But relax, you’ve now learned the bulk of the code you’ll need for the rest of the tutorial!

## Metrics available in NetworkX

When you start work on a new dataset, it’s a good idea to get a general sense of the data. The first step, described above, is to simply open the files and see what’s inside. Because it’s a network, you know there will be nodes and edges, but how many of each are there? What information is appended to each node or edge?

In our case, we have 165,885 nodes and 246,529 edges. These edges currently don’t have directions (that is, we're assuming a symmetric relationship between pages; this is wrong, as the links imply movement from one page to another but might not necessarily go back again). We've also added all available information as attributes to our network, i.e. node name (the slug), node ID, and link type.

These details inform what you can or should do with your dataset. Too few nodes (say, 15), and a network analysis is less useful than drawing a picture or doing some reading; too many (say, 15 million), and you should consider starting with a subset or finding a supercomputer. 

Ours appears to be just right, where we can leverage network analysis to make better use of the data. We could also take a subgraph for areas of particularly interest (a GOV.UK project team could make a sub-graph if they have a list of pages they want).

The network’s properties also guide your analysis. Because this network is currently undirected, your analysis must use metrics that require symmetric edges between nodes. For example, you can determine what communities pages find themselves in, but you can’t determine the directional routes through which information might flow along the network (you’d need a directed network for that; which we'll look to implement in another notebook). By using the symmetric, undirected relationships in this case, you’ll be able to find sub-communities and the pages who are important to those communities, a process that would be more difficult (though still possible) with a directed network.

NetworkX allows you to perform most analyses you might conceive, but you must understand the affordances of your dataset and realize some NetworkX algorithms are more appropriate than others.

## The Shape of the Network

After seeing what the dataset looks like, it’s important to see what the network looks like. They’re different things. The dataset is an abstract representation of what you assume to be connections between entities; the network is the specific instantiation of those assumptions. The network, at least in this context, is how the computer reads the connections you encoded in a dataset. 

A network has a topology, or a connective shape, that could be centralized or decentralized; dense or sparse; cyclical or linear. A dataset does not, outside the structure of the table it’s written in.

The network’s shape and basic properties will give you a handle on what you’re working with and what analyses seem reasonable. You already know the number of nodes and edges, but what does the network ‘look’ like? Do nodes cluster together, or are they equally spread out? Are there complex structures, or is every node arranged along a straight line?

In [ ]:
# Calculate the number of nodes (slugs/pages) that are linked to themselves (self loops)
G.number_of_selfloops()

We can plot our graph by uncommenting the code below; however, given the number of nodes/edges, this can be computationally expensive!

In [ ]:
# padding = dict(x=(-10, 10), y=(-10, 10))
# simple_graph = hv.Graph.from_networkx(G, nx.layout.circular_layout).redim.range(**padding)

# %opts Graph [width=400 height=400]
# simple_graph

## Taking a subgraph

Depending on your data and the query you ran to get it, your graph might be quite big and a bit slow to work with (as in our case). We can take a subgraph of G, and visualise this instead.

To create a subgraph, we could use a list/array of pages directly, as that's how our nodes are index. We also have a dictionary that maps node ID to the pages of interest (`node_id_dict`). 

Let's use this first method, using the `node_names` array to extract all the ones that contain the word "brexit", and then render some basic visualisations (this is an overly simplistic approach, and you would miss lots of journeys but is fine for our demonstration purposes).

In [ ]:
# Extract 'brexit' pages using a list comprehension
pages_of_interest = [name for name in node_names if "brexit" in name.lower()]
print("Example pages of interest:\n{}\n".format("\n".join(pages_of_interest[0:10])))
print("Number of pages of interest: {}".format(len(pages_of_interest)))

In [ ]:
# Select the nodes of interest, and create a subgraph
G_sub = G.subgraph(pages_of_interest)
G_sub.name = "{} for pages of interest".format(G.name)
print(nx.info(G_sub))

Now we should be able to visualise this smaller subgraph:

In [ ]:
%opts Graph [width=400 height=400]

simple_graph = hv.Graph.from_networkx(G_sub, nx.circular_layout)
simple_graph 

### Bundling graphs

The datashader library provides algorithms for bundling the edges of a graph and HoloViews provides convenient wrappers around the libraries.

In [ ]:
# We can use the %%opts ipython magic to set some display properties
%opts Graph [width=600 height=600 xaxis=None yaxis=None bgcolor="white" tools=["hover"]]

# Note: bundling graphs, takes a while
bundled = hd.bundle_graph(simple_graph)
bundled

### Datashading graphs

For graphs with a large number of edges we can datashade the paths and display the nodes separately. This loses some of the interactive features but will let you visualize quite large graphs:

In [ ]:
hd.datashade(bundled, normalization='linear', width=800, height=800) * bundled.nodes

### Applying selections

Alternatively, we can select the nodes and edges by an attribute that resides on either. In this case we will select the nodes and edges for a particular circle and then overlay just the selected part of the graph on the datashaded plot.

Note that selections on the Graph itself will select all nodes that connect to one of the selected nodes. In this way a smaller subgraph can be highlighted and the larger graph can be datashaded. 

In [ ]:
bundled.select(node_id_dict=24265, selection_mode='nodes')

### Prettification of subgraph

We could also refine further by following various blogs, such as this [one](http://stanmart.github.io/network/economics/visualization/holoviews/datashader/gephi/2018/10/27/econ_network_2/), but will leave this for now until we have more attributes in the data.

### Animating graphs 

Like all other elements Graph can be updated in a HoloMap or DynamicMap . Here we animate how the Fruchterman-Reingold force-directed algorithm lays out the nodes in real time.

In [ ]:
def get_graph(iteration):
    np.random.seed(10)
    return hv.Graph.from_networkx(G_sub, nx.spring_layout, iterations=iteration)

hv.HoloMap({i: get_graph(i) for i in range(5, 35, 5)},
           kdims='Iterations')

### Looks pretty, but only gets you so far...

There are lots of ways to visualise a network, and a force-directed layout, of which the above image is an example, is among the most common. 

Force-directed graphs attempt to find the optimum placement for nodes with a calculation based on the tension of springs in Hooke’s Law, which for smaller graphs often creates clean, easy-to-read visualisations. The visualisation embedded above shows you there is a single large component of connected nodes (in the center) and several small components with just one or two connections around the edges (this is to be expected as our page list is mainly for demo purposes; there's some Brexit pages that are picked up). 

This is a fairly common network structure. Knowing that there are multiple components in the network will usefully limit the calculations you’ll want to perform on it. By displaying the number of connections (known as degree, see below) as the size of nodes, the visualisation also shows that there are a few nodes with lots of connections that keep the central component tied together (although there's not any obvious hubs here). These large nodes are known as hubs, and the fact that they show up so clearly here gives you a clue as to what you’ll find when you measure centrality in the next section.

Visualisations, however, only get you so far. The more networks you work with, the more you realise most appear similar enough that it’s hard to tell one from the next. Quantitative metrics let you differentiate networks, learn about their topologies, and turn a jumble of nodes and edges into something you can learn from.

## Back to the bigger GOV.UK graph (size is only an issue for viz)

A good metric to begin with is network density. This is simply the ratio of actual edges in the network to all possible edges in the network. 

In an undirected network like this one, there could be a single edge between any two nodes, but as you saw in the visualization, only a few of those possible edges are actually present. Network density gives you a quick sense of how closely knit your network is.

And the good news is many of these metrics require simple, one-line commands in Python. From here forward, you can keep building on your code block from the previous sections. You don’t have to delete anything you’ve already typed, and because you created your network object `G` in the codeblock above, all the metrics from here on should work correctly.

You can calculate network density by running `nx.density(G)`. However the best way to do this is to store your metric in a variable for future reference, and print that variable, like so:

In [ ]:
density = nx.density(G)
round(density, 5)

density = nx.density(G_sub)
round(density, 5)

These numbers will vary based on your graph but the interpretation can be followed...

The output of density is a number, so that’s what you’ll see when you print the value. In this case, the density of our entire functional network is approximately 0.00002. On a scale of 0 to 1, not a very dense network. This contrasts with our "Brexit" subgraph which wasn't that dense but relatively 190 times moreso than the rest of GOV.UK. A 0 would mean that there are no connections at all, and a 1 would indicate that all possible edges are present (a perfectly connected network): this GOV.UK network is on the lower end of that scale, and close to 0. It would be interesting if we had a list of mainstream pages and compared density to the subgraph of Whitehall or specialist pages perhaps. 

A shortest path measurement is a bit more complex. It calculates the shortest possible series of nodes and edges that stand between any two nodes, something hard to see in large network visualisations. This measure is essentially finding friends-of-friends — if my mother knows someone that I don’t, then my mother is the shortest path between me and that person. The Six Degrees of Kevin Bacon game, from which our project takes its name, is basically a game of finding shortest paths (with a path length of six or less) from Kevin Bacon to any other actor.

To calculate a shortest path, you’ll need to pass several input variables (information you give to a Python function): the whole graph, your source node, and your target node. Let’s find the shortest path between two Brexit related pages. Note this might be more tricky with a directed graph, as not all pages may be connected due to the directionality of the edges.

Since we used names to uniquely identify our nodes in the network, you can access those nodes (as the source and target of your path), using the names directly. Or if the names are very long to type out, we might prefer to use node ID (from this [SO answer](https://stackoverflow.com/questions/8023306/get-key-by-value-in-dictionary) we get a key by a value in the dictionary).

In [ ]:
# Have some pages in mind, and find the shortest functional path
# I found the page using the visualisations, and looking at GOV.UK
pg1 = list(node_id_dict.keys())[list(node_id_dict.values()).index(156519)]
pg2 = list(node_id_dict.keys())[list(node_id_dict.values()).index(165861)]
print("Look for the shortest path between:\n'{0}' and '{1}'\n".format(pg1, pg2))

a_path_of_interest = nx.shortest_path(G, source=pg1, target=pg2)
print("Path is:\n{}".format("\n".join(a_path_of_interest)))

Depending on the size of your network, this could take a little while to calculate, since Python first finds all possible paths and then picks the shortest one. 

The output of the `shortest_path` function will be a list of the nodes that includes the “source”, the “target”, and the nodes between them. This isn't that informative here, but we might imagine if lots of shortest paths run through a particular page it could be considered a hub.

Python includes many tools that calculate shortest paths. There are functions for the lengths of shortest paths, for all shortest paths, and for whether or not a path exists at all in the documentation. You could use a separate function to find out the length of the Fell-Whitehead path we just calculated, or you could simply take the length of the list minus one (We take the length of the list minus one because we want the number of edges (or steps) between the nodes listed here, rather than the number of nodes.), like this:

In [ ]:
print("Length of that path:", len(a_path_of_interest)-1)

There are many network metrics derived from shortest path lengths. 

One such measure is diameter, which is the longest of all shortest paths. After calculating all shortest paths between every possible pair of nodes in the network, diameter is the length of the path between the two nodes that are furthest apart. The measure is designed to give you a sense of the network’s overall size, the distance from one end of the network to another.

Diameter uses a simple command: `nx.diameter(G)`. However, running this command on our graph will yield an error telling you the Graph is “not connected.” This simply means that your graph, as you already saw, has more than one component. Because there are some nodes that have no path at all to others, it is impossible to find all of the shortest paths. Take another look at the visualization of your graph:

In [ ]:
try:
    nx.diameter(G)  # might not work for directed graphs
except nx.NetworkXError as e:
    print("Error:\n{}".format(e))

Since there is no shortest path between nodes of one component and nodes of another, `nx.diameter()` returns the “not connected” error. You can remedy this by first finding out if your Graph “is connected” (i.e. all one component) and, if not connected, finding the largest component and calculating diameter on that component alone. Here’s the code:

Here’s the code for connectivity:

In [ ]:
# If your Graph has more than one component, this will return False:
print(nx.is_connected(G_sub))  # for undirected graphs only

# Next, use nx.connected_components to get the list of components, then use the max()
# command to find the largest one (for undirected graphs only):
components = nx.connected_components(G_sub)
largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component, then calculate the diameter of the subgraph, 
# just like you did with density - for undirected graphs only
subgraph = G_sub.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

Since we took the largest component, we can assume there is no larger diameter for the other components. Therefore this figure is a good stand in for the diameter of the whole Graph. The network diameter of this network’s largest strongly connected component is 12: there is a path length of 12 between the two farthest-apart nodes in the network. 

Unlike density which is scaled from 0 to 1, it is difficult to know from the network diameter number alone it is a large or small diameter. For some global metrics, it can be best to compare it to networks of similar size and shape. The most principled way of doing this kind of comparison is to create random graphs of identical size to see if the metrics differ from the norm. NetworkX offers plenty of tools for generating random graphs.

The final structural calculation you will make on this network concerns the concept of triadic closure. Triadic closure supposes that if two pages know the same page, they are likely to know each other. If C knows both A and B, then A and B may very well know each other, completing a triangle in the visualization of three edges connecting C, B, and C. The number of these enclosed triangles in the network can be used to find clusters and communities of individuals that all know each other fairly well.

One way of measuring triadic closure is called clustering coefficient because of this clustering tendency, but the structural network measure you will learn is known as transitivity (Why is it called transitivity? You might remember the transitive property from high school geometry: if A=B and B=C, the A must equal C. Similarly, in triadic closure, if person A knows page B and page B knows page C, then page A probably knows page C: hence, transitivity.). Transitivity is the ratio of all triangles over all possible triangles. A possible triangle exists when one page (C) knows two pages (A and B). So transitivity, like density, expresses how interconnected a graph is in terms of a ratio of actual over possible connections. Remember, measurements like transitivity and density concern likelihoods rather than certainties. All the outputs of your Python script must be interpreted, like any other object of research. Transitivity allows you a way of thinking about all the relationships in your graph that may exist but currently do not.

You can calculate transitivity in one line, the same way you calculated density:

In [ ]:
triadic_closure = nx.transitivity(G_sub)
print("Triadic closure:", triadic_closure)

Also like density, transitivity is scaled from 0 to 1, and you can see that the network’s transitivity is about 0.13, much higher than its 0.0038 density. Because the graph is not very dense, there are fewer possible triangles to begin with, which may result in slightly higher transitivity. That is, nodes that already have lots of connections are likely to be part of these enclosed triangles. To back this up, you’ll want to know more about nodes with many connections.

## Centrality

After getting some basic measures of the entire network structure, a good next step is to find which nodes are the most important ones in your network.

In network analysis, measures of the importance of nodes are referred to as centrality measures. Because there are many ways of approaching the question “Which nodes are the most important?” there are many different ways of calculating centrality. Here you’ll learn about three of the most common centrality measures: degree, betweenness centrality, and eigenvector centrality.

Degree is the simplest and the most common way of finding important nodes. A node’s degree is the sum of its edges. If a node has three lines extending from it to other nodes, its degree is three. Five edges, its degree is five. It’s really that simple. Since each of those edges will always have a node on the other end, you might think of degree as the number of pages to which a given page is directly connected. The nodes with the highest degree in a social network are the pages who know the most pages. These nodes are often referred to as hubs, and calculating degree is the quickest way of identifying hubs.

Calculating centrality for each node in NetworkX is not quite as simple as the network-wide metrics above, but it still involves one-line commands. All of the centrality commands you’ll learn in this section produce dictionaries in which the keys are nodes and the values are centrality measures. That means they’re ready-made to add back into your network as a node attribute, like you did in the last section. Start by calculating degree and adding it as an attribute to your network.

In [ ]:
# We can do it for the whole graph here
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

# For a directed graph, we can also add in and out degrees
# in_degree_dict = dict(G.in_degree(G.nodes()))
# out_degree_dict = dict(G.out_degree(G.nodes()))
# nx.set_node_attributes(G, in_degree_dict, 'in_degree')
# nx.set_node_attributes(G, out_degree_dict, 'out_degree')

You just ran the `G.degree()` method on the full list of nodes in your network (`G.nodes()`). Since you added it as an attribute, you can now see `gov.uk/`’s degree along with its other information if you access this node directly:

In [ ]:
print(G.node['/'])

But these results are useful for more than just adding attributes to your Graph object. Since you’re already in Python, you can sort and compare them. You can use the built-in function `sorted()` to sort a dictionary by its keys or values and find the top twenty nodes ranked by degree. To do this you’ll need to use itemgetter, which we imported back at the beginning of the tutorial. Using sorted and `itemgetter`, you can sort the dictionary of degrees like this:

In [ ]:
sorted_in_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print("Top 20 nodes by degree:\n{}".format("\n".join(str(node) for node in sorted_in_degree[:20])))

As you can see, `gov.uk/`’s degree is relatively low compared to other pages in this network, such as the `/government/publications/environmental-permitting-public-participation-statement` page. If we were looking at how users traverse GOV.UK (user journeys), we would expect the home page to be the most important page; in our case, we're looking at links on pages, and few pages link directly to the home page. 

Assuming you were looking at user journeys instead, you probably don’t need NetworkX to tell you that the home page is important in an undirected graph. Most social networks will have just a few hubs of very high degree, with the rest of similar, much lower degree. Degree can tell you about the biggest hubs, but it can’t tell you that much about the rest of the nodes. And in many cases, those hubs it’s telling you about are not especially surprising. In this case almost all of the hubs are mainstream content.

### Eigenvector centrality

Thankfully there are other centrality measures that can tell you about more than just hubs. Eigenvector centrality is a kind of extension of degree—it looks at a combination of a node’s edges and the edges of that node’s neighbors.

Eigenvector centrality cares if you are a hub, but it also cares how many hubs you are connected to. It’s calculated as a value from 0 to 1: the closer to one, the greater the centrality. Eigenvector centrality is useful for understanding which nodes can get information to many other nodes quickly. If you know a lot of well-connected pages, you could spread a message very efficiently. If you’ve used Google, then you’re already somewhat familiar with Eigenvector centrality. Their PageRank algorithm uses an extension of this formula to decide which webpages get to the top of its search results. We can use this metric on our subgraph to identify pages with lots of well-connected neighbours.

### Betweeness centrality

Betweenness centrality is a bit different from the other two measures in that it doesn’t care about the number of edges any one node or set of nodes has. Betweenness centrality looks at all the shortest paths that pass through a particular node (see above). To do this, it must first calculate every possible shortest path in your network, so keep in mind that betweenness centrality will take longer to calculate than other centrality measures (but it won’t be an issue in a dataset of this size). 

Betweenness centrality, which is also expressed on a scale of 0 to 1, is fairly good at finding nodes that connect two otherwise disparate parts of a network. If you’re the only thing connecting two clusters, every communication between those clusters has to pass through you. In contrast to a hub, this sort of node is often referred to as a broker. Betweenness centrality is not the only way of finding brokerage (and other methods are more systematic), but it’s a quick way of giving you a sense of which nodes are important not because they have lots of connections themselves but because they stand between groups, giving the network connectivity and cohesion.

These two centrality measures are even simpler to run than degree—they don’t need to be fed a list of nodes, just the graph `G`. You can run them with these functions:

In [ ]:
degree_dict = dict(G_sub.degree(G_sub.nodes())) # Rerun for Brexit subgraph
betweenness_dict = nx.betweenness_centrality(G_sub) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G_sub, max_iter=500) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G_sub, degree_dict, 'degree')
nx.set_node_attributes(G_sub, betweenness_dict, 'betweenness')
nx.set_node_attributes(G_sub, eigenvector_dict, 'eigenvector')

# Check they've been added
G_sub.node['/government/brexit']

In [ ]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)
print("Top 10 nodes by betweenness centrality:\n{}".format(
    "\n".join(str(b) for b in sorted_betweenness[:10])))

In [ ]:
sorted_eigenvector = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)

print("Top 10 nodes by eigenvector centrality:\n{}".format(
    "\n".join(str(e) for e in sorted_eigenvector[:10])))

You’ll notice that many, but not all, of the nodes that have high degree also have high betweenness centrality. In fact, betweeness centrality might surface some useful insights, especially if we include additional metadata as attributes for our nodes.

In [ ]:
# Get the top 10 notes by betweeness, and print out their degree
for tb in sorted_betweenness[:10]:
    degree = degree_dict[tb[0]]  # use degree_dict to access a node's degree
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

Pages that have a high betweenness centrality but low degree could be important brokers, connecting otherwise disparate parts of the graph.

## Advanced NetworkX: Community detection with modularity

Another common thing to ask about a network dataset is what the subgroups or communities are within the larger social structure. Is your network one big, happy family where everyone knows everyone else? Or is it a collection of smaller subgroups that are only connected by one or two intermediaries? The field of community detection in networks is designed to answer these questions.

There are many ways of calculating communities, cliques, and clusters in your network, but the most popular method currently is modularity. Modularity is a measure of relative density in your network: a community (called a module or modularity class) has high density relative to other nodes within its module but low density with those outside. Modularity gives you an overall score of how fractious your network is, and that score can be used to partition the network and return the individual communities.

Very dense networks are often more difficult to split into sensible partitions. Luckily, as you discovered earlier, this network is not all that dense. There aren’t nearly as many actual connections as possible connections, and there are several altogether disconnected components. Its worthwhile partitioning this sparse network with modularity and seeing if the result make historical and analytical sense.

Community detection and partitioning in NetworkX requires a little more setup than some of the other metrics. There are some built-in approaches to community detection (like minimum cut), but modularity is not included with NetworkX. Fortunately there’s an additional python module you can use with NetworkX, which you already installed and imported at the beginning of this tutorial. You can read the full documentation for all of the functions it offers, but for most community detection purposes you’ll only want `best_partition()`:

In [ ]:
communities = community.best_partition(G_sub)

The above code will create a dictionary just like the ones created by centrality functions. `best_partition()` tries to determine the number of communities appropriate for the graph, and assigns each node a number (starting at 0), corresponding to the community it’s a member of. You can add these values to your network in the now-familiar way:

In [ ]:
nx.set_node_attributes(G_sub, communities, "modularity")

In [ ]:
# First get a list of just the nodes in that class
class0 = [n for n in G_sub.nodes() if G_sub.node[n]["modularity"] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G_sub.node[n]["eigenvector"] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

Using eigenvector centrality as a ranking can give you a sense of the important pages within this modularity class. You’ll need some expert knowledge of the data to probably glean insights here, so it's worth talking to the programme team as to why these pages are clustered. With just a little bit of digging, we can get an indication that modularity is probably working as expected.

In smaller networks like this one, a common task is to find and list all of the modularity classes and their members. You can do this by manipulating the communities dictionary. You’ll need to reverse the keys and values of this dictionary so that the keys are the modularity class numbers and the values are lists of names. You can do so like this:

In [ ]:
modularity = {} # Create a new, empty dictionary
for k,v in communities.items():  # Loop through the community dictionary
    if v not in modularity:
        modularity[v] = [k]  # Add a new key for a modularity class the code hasn't seen before
    else:
        modularity[v].append(k)  # Append a name to the list for a modularity class the code has already seen

for k,v in modularity.items():  # Loop through the new dictionary
    if len(v) > 2: # Filter out modularity classes with 2 or fewer nodes
        print("Class {0}:\n{1}\n".format(k, v))  # Print out the classes and their members

Notice in the code above that you are filtering out any modularity classes with two or fewer nodes, in the line if `len(v) > 2`. You’ll remember from the visualization that there were lots of small components of the network with only two nodes. Modularity will find these components and treat them as separate classes (since they’re not connected to anything else). By filtering them out, you get a better sense of the larger modularity classes within the network’s main component. We should adjust this threshold accordingly, based on the specific problem and context of your question.

Working with NetworkX alone will get you far, and you can find out a lot about modularity classes just by working with the data directly. But you’ll almost always want to visualize your data (and perhaps to express modularity as node color). In the next section you’ll learn how to export your NetworkX data for use in other programs.

In [ ]:
print(G_sub.node["/government/brexit"])

## Exporting data

NetworkX supports a very large number of file formats for data export. If you wanted to export a plaintext edgelist to load into Palladio, there’s a convenient wrapper for that. Frequently at Six Degrees of Francis Bacon, we export NetworkX data in D3’s specialized JSON format, for visualization in the browser. You could even export your graph as a Pandas dataframe if there were more advanced statistical operations you wanted to run. There are lots of options, and if you’ve been diligently adding all your metrics back into your Graph object as attributes, all your data will be exported in one fell swoop.

Most of the export options work in roughly the same way, so for this tutorial you’ll learn how to export your data into Gephi’s GEXF format. Once you’ve exported the file, you can upload it directly into Gephi for visualization.

Exporting data is often a simple one-line command. All you have to choose is a filename. In this case we’ll use `brexit_network.gexf`. To export type:

That’s it! When you run your Python script, it will automatically place the new GEXF file in the same directory as your Python file.

In [ ]:
# Our brexit subgraph (remember this is not all Brexit content, just with Brexit in the url)
print(nx.info(G_sub))
# nx.write_gexf(G, 'brexit_network.gexf')

## Conclusion

In this tutorial we read in data as an adjacency list, converted it to a node and edge list.
 We created a graph object with the networkx and then calculated a variety of metrics that could be added as attributes of the nodes.
  Likewise we could use a similar approach for the edges if so inclined. We also used some pyviz modules to visualise the topology of a network.

Representing data as a graph or network is powerful particularly when combined with machine learning.  